In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [4]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [5]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [6]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [7]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

東京都と沖縄県を面積で比較すると、沖縄県の方が広いです。具体的な面積は以下の通りです。

- 沖縄県の面積: 約2,276.64 km²
- 東京都の面積: 約2,194.97 km²

したがって、沖縄県が東京都よりも広いことがわかります。


In [8]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

最近1ヶ月以内の東京駅周辺のイベント情報は以下の通りです。

1. **[東京駅周辺のイベント](https://www.walkerplus.com/event_list/ar0313/sc309880d/)**
   - 開催日: 2025年3月20日（土）〜3月3日（火）
   - 内容: CREATIVE MUSEUM TOKYOなどでのイベント。

2. **[イプンテレポート - 東京街人](https://guidetokyo.info/join/report/index.html)**
   - 開催期間: 2023年10月3日から31日まで
   - 内容: 東京駅前に「YAESU st. PARKLET（弥生桜の公園）」が登場。

3. **[超！海士町祭り - Beyondアマフェス](https://note.com/beyond_ama_fes/n/ne6aa4513738c)**
   - 開催日: 11月9日（土）
   - 内容: 東京の日比谷で行われる祭り。

4. **useums.tokyo](https://6museums.tokyo/)**
   - 開催: 東京駅周辺の6つの美術館での文化・芸術関連のイベント。

これらのイベントは、東京駅及びその周辺で行われており、美術展や地域イベントなど多岐にわたります。興味のあるイベントがあれば、リンクを参照して詳細を確認してください。


In [9]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは'

こんにちは！今日はどんなことに興味がありますか？


'質問:東京駅で今やっているイベントを検索して'

東京駅で現在開催されているイベントについての情報をいくつか紹介します。

1. **東京駅周辺のイベント**
   - **開催期間**: 2025年4月20日(日)～6月3日(火)
   - **詳細**: 東京駅周辺でさまざまなイベントが開催されています。[詳細はこちら](https://www.walkerplus.com/event_list/ar0313/sc309880d/)

2. **東京駅周辺のアートイベント**
   - **開催期間**: 2025年4月1日(土)～6月3日(火)
   - **詳細**: アートに関するイベントが開催されています。[詳細はこちら](https://ekitan.com/event/station-2590)

3. **東京駅での特別展**
   - **開催期間**: 2025年5月6日(月)～9月30日(日)
   - **詳細**: 特別な展示や企画が行われています。[詳細はこちら](https://travel.biglobe.ne.jp/family/event/area/1301_1.html)

4. **東京駅直結のビアフェスタ**
   - **開催期間**: 2025年6月13日(金)～6月26日(日)
   - **詳細**: 東京駅付近でビールを楽しむイベントです。[詳細はこちら](https://www.enjoytokyo.jp/event/list/sta200101/its04/)

これらのイベントは、東京の文化や楽しい体験を提供しているものです。興味があるイベントがあれば、リンクをクリックして詳細を確認してみてください！


'質問:東京駅の住所は'

東京駅の住所は「東京都千代田区丸の内1丁目9-1」です。この場所にはJR東日本の東京駅があり、多くの鉄道路線が交差する重要な交通ハブとなっています。さらに詳しい情報は、[こちらの記事](https://ja.wikipedia.org/wiki/%E6%9D%B1%E4%BA%AC%E9%A7%85)をご覧ください。

---ご利用ありがとうございました！---
